In [1]:
import io
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
conn = sqlite3.connect('sql/conn_trabalho.db', timeout=10)
c = conn.cursor()

In [3]:
dataB = pd.read_csv("dataset/LGBT_Survey_DailyLife.csv")
dataB.head()

,CountryCode,subset,question_code,question_label,answer,percentage,notes
0,Austria,Lesbian,b1_a,"In your opinion, how widespread is offensive l...",Very widespread,8,NaN
1,Austria,Lesbian,b1_a,"In your opinion, how widespread is offensive l...",Fairly widespread,34,NaN
2,Austria,Lesbian,b1_a,"In your opinion, how widespread is offensive l...",Fairly rare,45,NaN
3,Austria,Lesbian,b1_a,"In your opinion, how widespread is offensive l...",Very rare,9,NaN
4,Austria,Lesbian,b1_a,"In your opinion, how widespread is offensive l...",Don`t know,4,[0]


In [4]:
dataB["CountryCode"]

0        Austria
1        Austria
2        Austria
3        Austria
4        Austria
          ...   
34015    Average
34016    Average
34017    Average
34018    Average
34019    Average
Name: CountryCode, Length: 34020, dtype: object

# Criando as tabelas

In [5]:
c.execute('''CREATE TABLE COUNTRY
             (COUNTRYCODE         INT NOT NULL,
              COUNTRY             TEXT NOT NULL,
              
              PRIMARY KEY(COUNTRYCODE)
              
             )''')

In [6]:
c.execute('''CREATE TABLE QUESTIONS
             (QC                  VARCHAR(10) NOT NULL,
              QUESTION            TEXT NOT NULL,
              CATEGORY            TEXT NOT NULL,
              
              PRIMARY KEY(QC)
             )''')

In [7]:
c.execute('''CREATE TABLE ANSWERS
             (COUNTRYCODE         INT NOT NULL,
              SUBSET              TEXT NOT NULL,
              QC                  VARCHAR(10) NOT NULL,
              ANSWER              TEXT NOT NULL,
              PERCENTAGE          INT NOT NULL,
              NOTES               TEXT,
              
              PRIMARY KEY(COUNTRYCODE,SUBSET,QC,ANSWER),
              
              FOREIGN KEY(COUNTRYCODE) REFERENCES COUNTRY(COUNTRYCODE),
              FOREIGN KEY(QC) REFERENCES QUESTIONS(QC)
              
             )''')

In [8]:
c.execute('''CREATE TABLE SIZE
             (COUNTRYCODE         INT NOT NULL,
              LESBIAN             INT NOT NULL,
              GAY                 INT NOT NULL,
              BIW                 INT NOT NULL,
              BIM                 INT NOT NULL,
              TRANS               INT NOT NULL,
              
              PRIMARY KEY(COUNTRYCODE),
              
              FOREIGN KEY(COUNTRYCODE) REFERENCES COUNTRY(COUNTRYCODE)
              
             )''')

In [35]:
c.execute('''CREATE TABLE IDH
             (COUNTRYCODE         INT NOT NULL,
              HDI                 REAL NOT NULL,
              LEB                 REAL NOT NULL,
              EYS                 REAL NOT NULL,
              MYS                 REAL NOT NULL,
              GNI                 REAL NOT NULL,
              GNI_HDI             REAL NOT NULL,
              
              PRIMARY KEY(COUNTRYCODE),
              
              FOREIGN KEY(COUNTRYCODE) REFERENCES COUNTRY(COUNTRYCODE)
              
             )''')

# Variaveis auxiliares

In [9]:
conv = open("conv.txt","r")

table1 = {}

p = conv.read().split("\n")

for x in p:
    s = x.split("|")[0][:-1].upper()
    i = x.split("|")[1][1:].upper()
    
    table1[i] = s

In [10]:
tableS = {"Lesbian": "LESBIAN",
          "Gay": "GAY",
          "Bisexual women" : "BIW",
          "Bisexual men" : "BIM",
          "Transgender" : "TRANS"}


# Populando

In [11]:
#Country

table2 = {}

data = pd.read_csv("dataset/LGBT_Survey_SubsetSize.csv")

data = data.drop(0)

data = data.sort_values("CountryID")

paises = data["CountryID"].unique()

for x in range(len(paises)):
    
    table2[paises[x]] = x
    
    querry = """INSERT INTO COUNTRY
          VALUES ({},'{}') """.format(x,paises[x])
    
    c.execute(querry)

In [16]:
#questions

code = []
for x in os.listdir("dataset"):    
    data = pd.read_csv("dataset\{}".format(x))
    
    if list(data.columns).count("question_code") == 0:
        continue
    
    cat = x.split("_")[2]
    cat = cat.split(".")[0]
    
    q = data[["question_code","question_label"]].values
    
    for i in q:
        if code.count(i[0]):
            continue
        
        code.append(i[0])
        
        querry = """INSERT INTO QUESTIONS
          VALUES ('{}','{}','{}') """.format(i[0],i[1],cat)
        
        c.execute(querry)
    

.~lock.human-development-index-hdi-2014.csv#
human-development-index-hdi-2014.csv
LGBT_Survey_DailyLife.csv
LGBT_Survey_Discrimination.csv
LGBT_Survey_RightsAwareness.csv
LGBT_Survey_SubsetSize.csv
LGBT_Survey_TransgenderSpecificQuestions.csv
LGBT_Survey_ViolenceAndHarassment.csv


In [17]:
#Answers

for x in os.listdir("dataset"):
    data = pd.read_csv("dataset\{}".format(x))
    
    if list(data.columns).count("question_code") == 0:
        continue
        
    data = data.fillna("NULL")
    
    for i in data.index:
        sample = data.loc[i]
        
        try:
            idC = table1[sample["CountryCode"].upper()]
            idC = table2[idC]
            
        except:
            continue
        
        subset = tableS[sample["subset"]]
        qc = sample["question_code"]
        answer = sample["answer"]
        percentage = sample["percentage"]
        notes = sample["notes"]
        
        querry = """INSERT INTO ANSWERS
          VALUES ({},'{}','{}','{}',{},'{}') """.format(idC,subset,qc,answer,percentage,notes)
        
        try:
            c.execute(querry)            
        except:
            continue

In [18]:
#SIZE

data = pd.read_csv("dataset/LGBT_Survey_SubsetSize.csv")

for x in data.index:
    sample = data.loc[x]
    
    idC = sample["CountryID"]
    
    try:
        idC = table2[idC]
    except:
        continue
    
    
    les = sample["Lesbian women"]
    gay = sample["Gay men"]
    biW = sample["Bisexual women"]
    biM = sample["Bisexual men"]
    trans = sample["Transgender"]
    
    querry = """INSERT INTO SIZE
          VALUES ({},{},{},{},{},{}) """.format(idC,les,gay,biW,biM,trans)
    
    c.execute(querry)



In [59]:
#IDH

data = pd.read_csv("dataset/human-development-index-hdi-2014.csv")

for x in range(data.shape[0]):
    sample = data.loc[x]
    
    country = sample["Location"]
    
    if list(table1.keys()).count(country.upper()) == 0:
        continue
    
    idC = table1[country.upper()]
    idC = table2[idC]
    
    sample = sample.values
    sample = sample[2:]
    
    querry = """INSERT INTO IDH
          VALUES ({}""".format(idC)
    
    for y in sample:
        if type(y) == str:
            y = y.split(",")
            y = ".".join(y)
            y =  float(y)
            
        querry += ",{}".format(y)
    
    querry += ")"
    
    c.execute(querry)

# Salvando

In [60]:
#backup
#Referencia: http://pythonclub.com.br/gerenciando-banco-dados-sqlite3-python-parte1.html, https://stackoverflow.com/questions/49394737/exporting-data-from-google-colab-to-local-machine

f = io.open("sql/trabalho.sql",'w')

for texto in conn.iterdump():
    f.write("{}\n".format(texto))

f.close()

In [ ]:
#conn.close()